# Compare Inventory ananlysis results in different period
1. Find diffence bet two distribution reports.
2. Check robust. (How many common SKU are there). 


## Questions:
### Distrubution report
1. How many (%) SKU has different facing or capacity in these two period?
   - 125/679 = **18%**
1. what reason leads to adjust in distribution report?

### Full Month SKU
1. The percentage of SKUs (full month), in both period? 
   - 74/110 = **67%**
2. **Does fullmonth SKU (based on July-Sep) capacity decrease in new distribution report?**
   - There are 55 SKU's capacity decrease from July to Sep, 
      1. only 8 of them are shown in fullMonthSKU in July
      2. 7 of them are shown in fullMonthSKU in two period.
   - (`Incorrect decision`): There are 68 SKU's capacity increase from July to Sep, 
      1. 17 of them are shown in fullMonthSKU in July. 
      2. 8 of them are shown in fullMonthSKU in two period.
   - Thus, the current Capacity adjustment decision has no strong relationship with Qtysold of previous month.

3. **For the fullmonth SKU, which in one period only, does them appear in atLeastoneMonthSKU of another period?**

### Issued SKU

In [2]:
import pandas as pd
from pandas import ExcelWriter
from datetime import datetime, timedelta
import warnings
warnings.filterwarnings('ignore')
from Inventory_opti_helperFunction import *

In [7]:
# Python code t get difference of two lists 
# Using set() 
def Diff(li1, li2): 
    return (list(set(li1) - set(li2))) 
  
# Driver Code 
li1 = [10, 15, 20, 25, 30, 35, 40] 
li2 = [25, 40, 35, 34343] 
print(Diff(li1, li2))

[10, 20, 30, 15]


### Compare two distribution report

In [28]:
dist_df_Sep  = pd.read_csv("../data/RawData/Distribution Report - Auckland Departures - July19.csv")
dist_df_Jul  = pd.read_csv("../data/RawData/Distribution Report- Auckland Departures - Jan2020.csv")
dist_df_Sep = dist_df_Sep[['Mat ID', 'Name', '# POGs', 'Facings', 'Capacity']]
dist_df_Jul = dist_df_Jul[['Mat ID', 'Name', '# POGs', 'Facings', 'Capacity']]

In [62]:
125/679

0.18409425625920472

In [58]:
dist_df_adjust = pd.merge(dist_df_Jul, dist_df_Sep, on=['Mat ID', 'Name'])
dist_df_adjust = dist_df[(dist_df['Capacity_x']!=dist_df['Capacity_y']) | (dist_df['Facings_x']!=dist_df['Facings_y'])]
print(len(dist_df_adjust))
dist_df_adjust.head()

125


,Mat ID,Name,# POGs_x,Facings_x,Capacity_x,# POGs_y,Facings_y,Capacity_y
1,100078996,APEROL 100CL 11%,1,3,12,1,2,8
3,10389,BAILEYS IRISH CREAM 100CL 17%,3,42,248,2,39,236
11,52812,CAMPARI BITTER 100CL 28.5%,1,3,12,1,2,8
12,100366392,CHAMBORD BLACK 70CL 16.5%,1,6,18,1,2,6
13,10316,COINTREAU 100CL 40%,3,16,54,2,13,42


### Compare full month SKU

In [78]:
df_SepNov  = pd.read_csv("../data/Output/fullMonth/qty_less_than_capacity_all_month_SKU.csv")
df_JulSep  = pd.read_csv("../data/Output_July/fullMonth/qty_less_than_capacity_all_month_SKU.csv")

In [68]:
SKU_SepNov  = df_SepNov['SKU'].unique()
SKU_JulSep  = df_JulSep['SKU'].unique()

print("{} Unique SKU (fullmonth) in SepNov".format(len(SKU_SepNov)))
print("{} Unique SKU (fullmonth) in JulSep".format(len(SKU_JulSep)))

110 Unique SKU (fullmonth) in SepNov
117 Unique SKU (fullmonth) in JulSep


In [69]:
# Python code t get difference of two lists 
# Using set() 
def Diff(li1: list, li2:list) -> list:
    """
    get the different item shown in li1, not li2
    """
    return (list(set(li1) - set(li2))) 

#### Common SKU in two period

In [71]:
SKU_common = list(set(SKU_SepNov).intersection(SKU_JulSep))
print("{} Unique SKU (fullmonth) in SepNov and JulSep".format(len(SKU_common)))

74 Unique SKU (fullmonth) in SepNov and JulSep


In [20]:
74/110

0.6727272727272727

In [95]:
SKU_common_df = pd.DataFrame(SKU_common , columns = ['Name']) 

#### Difference bet two period
check if these items 

In [73]:
SKU_inSepNot_notJulSep = Diff(SKU_SepNov_list, SKU_JulSep_list)

In [72]:
SKU_inJulSep_notSepNov = Diff(SKU_JulSep_list, SKU_SepNov_list)

### Question: Does fullmonth SKU (based on July-Sep) capacity decrease in new distribution report?

There are 55 SKU's capacity decrease from July to Sep, but only 8 of them are shown in fullMonthSKU in July.

In [100]:
dist_df_capacity_decrease = dist_df[(dist_df['Capacity_x'] > dist_df['Capacity_y'])]
print(len(dist_df_capacity_decrease))

# check items which are fullMonthS SKU in July
check_fullMonth_july = pd.merge(dist_df_capacity_decrease, df_JulSep, left_on = ['Mat ID', 'Name'], 
                 right_on=['MatID', 'SKU'], how='inner')
print(len(check_fullMonth_july['MatID'].unique())) # number of items find 

# check items which are fullMonthS SKU in 2 period
check_common =  pd.merge(dist_df_capacity_decrease, SKU_common_df, on = 'Name', how='inner')
print(len(check_common['Name'].unique())) # number of items find

55
8
7


In [101]:
dist_df_capacity_increase = dist_df[(dist_df['Capacity_x'] < dist_df['Capacity_y'])]
print(len(dist_df_capacity_increase))

check = pd.merge(dist_df_capacity_increase, df_JulSep, left_on = ['Mat ID', 'Name'], 
                 right_on=['MatID', 'SKU'], how='inner')
print(len(check['MatID'].unique())) # number of items find 

# check items which are fullMonthS SKU in 2 period
check_common =  pd.merge(dist_df_capacity_increase, SKU_common_df, on = 'Name', how='inner')
print(len(check_common['Name'].unique())) # number of items find

68
17
8
